In [75]:
# load data and write out sentence and target
import pandas as pd

loaded_set = pd.read_excel("Dataset/"+"training.xlsx")


data_set

,sentence,target
0,Weitergehende Sicherungsmaßnahmen können eine ...,3.800000
1,In der großen Kasernenanlage im Norden Kiels k...,2.285714
2,Premierminister David Lloyd George honorierte ...,3.000000
3,Der Beitrag der Truppen dieser Dominions währe...,4.000000
4,Eine Balance zwischen verschiedenen Lebensbere...,4.900000
...,...,...
895,"Zwischen 1901 und 2010 ist er um ca 1,7 cm pro...",3.375000
896,Aus Furcht vor einem Bürgerkrieg wollte sie – ...,4.058824
897,In den meisten Politikfeldern gilt dafür seit ...,5.111111
898,"Aufgrund der Wärmekapazität des Gesteins, und ...",5.000000


In [10]:
import tensorflow_hub as hub
from transformers import AutoModel, AutoTokenizer
# german tokens for bert
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-german-cased")
#model = AutoModel.from_pretrained("dbmdz/bert-base-german-cased")

Downloading: 100%|██████████| 456/456 [00:00<00:00, 228kB/s]
Downloading: 100%|██████████| 240k/240k [00:00<00:00, 559kB/s]
Downloading: 100%|██████████| 59.0/59.0 [00:00<00:00, 59.0kB/s]


In [76]:
# get the token_ids and the length of the token_ids of each sentence
token = []
token_ids = []
num_tokens = []
for x in range(len(loaded_set['Sentence'])):
    token.append(tokenizer.tokenize(loaded_set['Sentence'][x]))
    token_ids.append(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(loaded_set['Sentence'][x])))
    num_tokens.append(len(token_ids[x]))


900

In [105]:
# pad the token_ids with zeros so all of the vec have equal length 
import numpy as np
eq_token_id = []
for x in range(len(token_ids)):
    eq_token_id.append(np.asarray(np.concatenate([token_ids[x], np.zeros(max(num_tokens)-len(token_ids[x]))])))  


In [108]:
# create data frame and seperate in test and training 
from sklearn.model_selection import train_test_split

data_dict ={'word_vec': eq_token_id, 'target':loaded_set['MOS']}
data_set = pd.DataFrame.from_dict(data_dict)
y = data_set['target']
X = data_set.drop(['target'], axis=1)

X_train, X_test,  y_train, y_test = train_test_split(X, y, test_size=.2)


,word_vec
241,"[229.0, 7564.0, 113.0, 11835.0, 5319.0, 7416.0..."
851,"[286.0, 6845.0, 484.0, 6441.0, 231.0, 696.0, 9..."
576,"[928.0, 369.0, 195.0, 5946.0, 28199.0, 21404.0..."
665,"[465.0, 249.0, 1140.0, 125.0, 24448.0, 263.0, ..."
831,"[229.0, 5334.0, 818.0, 445.0, 1951.0, 12444.0,..."
...,...
565,"[1209.0, 13672.0, 728.0, 119.0, 125.0, 9281.0,..."
115,"[3463.0, 128.0, 1659.0, 199.0, 17456.0, 788.0,..."
765,"[2478.0, 15312.0, 8231.0, 313.0, 816.0, 18655...."
498,"[347.0, 4560.0, 2231.0, 683.0, 498.0, 159.0, 1..."


In [109]:
# set up a 1d CNN
import tensorflow as tf
# setup sequential modell 
model = tf.keras.Sequential()
# do a 1d convolution on the tolekn_ids
model.add(tf.keras.layers.Conv1D(filters=10, kernel_size=10, padding='same', activation='relu',input_shape=(max(num_tokens), 1)))
# maxpool the results
model.add(tf.keras.layers.MaxPool1D(pool_size=4))
# add LSTM
model.add(tf.keras.layers.LSTM(64))
# add dense layer 
model.add(tf.keras.layers.Dense(units=1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error', metrics='acc')

In [110]:
model.fit(X_train, y_train, epochs=100)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).